<a href="https://colab.research.google.com/github/BrunoBmassa/Python-Actitivies/blob/main/tratamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [56]:
import pyspark
import gdown
from IPython import get_ipython
from IPython.display import display
from pyspark.sql import SparkSession
from pyspark.sql.types import*
from pyspark.sql.functions import date_format, col, count, when, regexp_extract
from datetime import date
from os.path import join

In [65]:
spark= SparkSession.builder.getOrCreate()

In [ ]:
#Leia o arquivo ‘videos-stats.csv'  no dataframe 'df_video' com cabeçalho e inferindo o esquema
df_video_url='https://drive.google.com/file/d/1XAefeuunm6q7o0OEdF--NzEPXTmIG9i5/view?usp=drive_link'
# Extração do arquivo pela URL
file_id = df_video_url.split('/')[-2]
# Download do arquivo
output_file = 'videos-stats.csv'
gdown.download(id=file_id, output=output_file, quiet=False)
#Leitura do arquivo inferindo o esquema
df_video= spark.read.csv(output_file, header=True, inferSchema=True)
df_video.printSchema()


In [ ]:
#Altere os valores nulos dos campos 'Likes', 'Comments' e 'Views' para o valor 0

df_video = df_video.fillna({'Likes': 0, 'Comments': 0, 'Views': 0})
df_video.show()

In [ ]:
#Leia o arquivo ‘comments.csv' no dataframe 'df_comentario' com cabeçalho e inferindo o esquema

df_comentario_url='https://drive.google.com/file/d/1rW-4ufh6UqafMAeRrH1MVypHkS75lfFM/view?usp=drive_link'

# Extração do arquivo pela URL
file_id2 = df_comentario_url.split('/')[-2]

# Download do arquivo
output_file2 = 'comments.csv'
gdown.download(id=file_id2, output=output_file2, quiet=False)

#Leitura do arquivo
df_comentario = spark.read.csv(output_file2, header=True, inferSchema=True)
df_comentario.printSchema()

In [ ]:
#Calcule a quantidade de registros do df_video e df_comentario

df_video_contagem  = df_video.count()
df_comentario_contagem = df_comentario.count()

print("Quantidade de registros em df_video:", df_video_contagem)
print("Quantidade de registros em df_comentario:", df_comentario_contagem)

In [ ]:
#Remova os registros do df_video e df_comentario quem possuem o campo 'Video ID' nulos e calcule novamente a quantidade de registros

df_video = df_video.dropna(subset=['Video ID'])
df_comentario = df_comentario.dropna(subset=['Video ID'])

df_video_contagem  = df_video.count()
df_comentario_contagem = df_comentario.count()

print("Quantidade de registros em df_video:", df_video_contagem)
print("Quantidade de registros em df_comentario:", df_comentario_contagem)

In [ ]:
#Remova os registros apenas do df_video quem possuem o campo 'Video ID' duplicados

df_video = df_video.dropDuplicates(['Video ID'])
df_video_contagem  = df_video.count()

print("Quantidade de registros em df_video:", df_video_contagem)

In [44]:
#Converta os campos Likes, Comments e Views para 'int' no dataframe df_video

df_video = df_video.withColumn("Likes", df_video["Likes"].cast(IntegerType()))

In [45]:
#Converta os campos Likes e Sentiment para 'int' no dataframe df_comentario, além disso, altere o nome do campo Likes para 'Likes Comment'

df_comentario = df_comentario.withColumn("Likes", df_comentario["Likes"].cast(IntegerType()))
df_comentario = df_comentario.withColumn("Sentiment", df_comentario["Sentiment"].cast(IntegerType()))
df_comentario = df_comentario.withColumnRenamed("Likes", "Likes Comment")

In [ ]:
#Crie o campo 'Interaction' no dataframe df_video, com a soma dos campos Likes, Comments e Views

df_video = df_video.withColumn("Interaction", df_video["Likes"] + df_video["Comments"] + df_video["Views"])
print(df_video.show())

In [47]:
#Converta os campos 'Published At' para 'date' no dataframe df_video

df_video = df_video.withColumn("Published At", df_video["Published At"].cast(DateType()))

In [48]:
#Crie o campo 'Year' no dataframe df_video, extraindo apenas o ano do campo 'Published At'

df_video = df_video.withColumn("Year", date_format(df_video["Published At"], "yyyy"))

In [52]:

#Mescle os dados df_comentario no dataframe df_video em relação ao campo Video ID e crie o dataframe df_join_video_comments

mescla_video_comentario = join("df_video", "df_comentario")
df_join_video_comments = df_video.join(df_comentario, on="Video ID", how="left")


In [ ]:

#Leia o arquivo ‘videos-stats.csv'  no dataframe 'df_video' com cabeçalho e inferindo o esquema
df_us_videos_url='https://drive.google.com/file/d/1NmT0RoxSv9GSJ08qDIMdeRkwbsaXzJE_/view?usp=drive_link'
# Extração do arquivo pela URL
file_id3 = df_us_videos_url.split('/')[-2]
# Download do arquivo
output_file3 = 'USvideos.csv.csv'
gdown.download(id=file_id3, output=output_file3, quiet=False)
#Leitura do arquivo inferindo o esquema
df_us_videos= spark.read.csv(output_file3, header=True, inferSchema=True)
df_us_videos.printSchema()


In [ ]:
#Mescle os dados df_us_videos no dataframe df_video em relação ao campo Title e crie e visualize o dataframe df_join_video_usvideos

mescla_us_videos_video = join("df_video", "df_us_videos")
df_join_video_usvideos = df_video.join(df_us_videos, on="Title", how="left")
df_join_video_usvideos.show()


In [ ]:
#Verifique a quantidade de campos nulos em todos os campos do dataframe df_video

df_video_nulos = df_video.select([count(when(col(c).isNull(), c)).alias(c) for c in df_video.columns])
df_video_nulos.show()

In [63]:
#Remova a coluna '_c0' e salve o dataframe df_video como 'videos-tratados-parquet' no formato parquet e adicione o cabeçalho nos dados

df_video = df_video.drop("_c0")
df_video.write.option('header','true').mode("overwrite").parquet('videos-parquet')


In [ ]:
#Remova a coluna '_c0' e salve o dataframe df_join_video_comments como 'videos-comments-tratados-parquet' no formato parquet e adicione o cabeçalho nos dados

df_join_video_comments = df_join_video_comments.drop("_c0")
df_join_video_comments.write.option('header','true').mode("overwrite").parquet('videos-comments-parquet')